<a href="https://colab.research.google.com/github/JavierPerez21/QHack2022/blob/master/Coding_Challenges/games_100_TardigradeMasquerade_template/games_100_TardigradeMasquerade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install pennylane

In [ ]:
import pennylane as qml
from pennylane import numpy as np

In [this paper](https://arxiv.org/abs/2112.07978) the authors demonstrate to have entangled superconducting qubits and a tardigrade. The goal of this challenge is to replicate some of their results.

First we need to create two states. One without the tardigrade:

$$
|\phi\rangle_{AB} = \frac{1}{\sqrt{2}} ( |0\rangle_A \otimes |1\rangle_B + |1\rangle_A \otimes|0\rangle_B)
$$

And one with the tardigrade:


$$
|\phi\rangle_{ABT} = \frac{1}{\sqrt{2}} ( |0\rangle_A \otimes |e\rangle_{BT} + |1\rangle_A \otimes|g0\rangle_{BT})
$$
with
$$
|e\rangle_{BT} = cos \frac{\theta}{2}|1\rangle_B\otimes|0\rangle_T + sin \frac{\theta}{2}|0\rangle_B\otimes|1\rangle_T \;\;\; 
|g\rangle_{BT} = |0\rangle_B\otimes|0\rangle_T
$$

To demonstrate the existence of entanglement due to the tardigrade we need to calculate a measuere of [entanglement entropy](https://en.wikipedia.org/wiki/Entropy_of_entanglement) known as [Rény entropy](https://en.wikipedia.org/wiki/R%C3%A9nyi_entropy) with respect to qubit B

$$
S_2 (\rho_B) = - ln Tr(\rho^2_B)\;\;\;\; with\;\;\;\;  \rho_B = Tr_{A, T}(\rho_{ABT})\;\;\;\; and \;\;\;\; \rho_{ABT} = |\psi\rangle\langle\psi|_{ABT}
$$

This value achieves a maximum when $\rho_B=1/2I$ is maximally mixed. This maximum value os 0.693 happens when there is no tardigrade: $\mu_{AB} = |\phi\rangle\langle\phi|_{AB}$

The goal in this challenge is to prepare these two states and measure their renyi entropy.

In [ ]:

def second_renyi_entropy(rho):
    """Computes the second Renyi entropy of a given density matrix."""
    # DO NOT MODIFY anything in this code block
    rho_diag_2 = np.diagonal(rho) ** 2.0
    return -np.real(np.log(np.sum(rho_diag_2)))


def compute_entanglement(theta):
    """Computes the second Renyi entropy of circuits with and without a tardigrade present.

    Args:
        - theta (float): the angle that defines the state psi_ABT

    Returns:
        - (float): The entanglement entropy of qubit B with no tardigrade
        initially present
        - (float): The entanglement entropy of qubit B where the tardigrade
        was initially present
    """

    dev = qml.device("default.qubit", wires=3)

    # QHACK #
    G = np.identity(4)
    G[1, 1] = np.cos(theta / 2)
    G[1, 2] = np.sin(theta / 2)
    G[2, 1] = -np.sin(theta / 2)
    G[2, 2] = np.cos(theta / 2)

    # QHACK #
    @qml.qnode(dev)
    def circuit_ab():
        # Prepare maximally entangled state
        qml.Hadamard(wires=0)
        qml.PauliX(wires=1)
        qml.CNOT(wires=[0, 1])
        # return qml.state()
        return qml.density_matrix([1])

    @qml.qnode(dev)
    def circuit_abt():
        # Prepare maximally entangled state
        qml.Hadamard(wires=0)
        qml.PauliX(wires=1)
        qml.CNOT(wires=[0, 1])
        # Add tardigrade entanglement
        qml.QubitUnitary(G, wires=[1, 2])
        return qml.density_matrix([1])

    return second_renyi_entropy(circuit_ab()), second_renyi_entropy(circuit_abt())

Testing with 1.in

In [ ]:
theta = np.array(0.7745598277086928, dtype=float)

S2_without_tardigrade, S2_with_tardigrade = compute_entanglement(theta)
print(*[S2_without_tardigrade, S2_with_tardigrade], sep=",")